# Show plot based on lacation and the happiness score/% of population with depression (side by side) marked by color intensity

# Show plot based on lacation and the happiness score marked by color intensity

# Show plot based on lacation and the % of Population with Depression score marked by color intensity